In [ ]:
##### 17-09-2024 The code below is a combination of already existing code by R. Buring, and adjustments made by F. Schipper 
# (particularly, the biggest adjustments are checking that the basis element(s) of the cocycle space are Hamiltonian, 
# commentary and printing of output). It is supplementary material accompanying proceedings written for the ISQS28 (Integrable 
#Systems and Quantum Symmetries) conference (see also: arxiv link??).

# We import the following (see https://github.com/rburing/gcaops) to be able to run the code.
from gcaops.graph.formality_graph import FormalityGraph
from gcaops.algebra.differential_polynomial_ring import DifferentialPolynomialRing
from gcaops.algebra.superfunction_algebra import SuperfunctionAlgebra
from gcaops.graph.undirected_graph_complex import UndirectedGraphComplex
from gcaops.graph.directed_graph_complex import DirectedGraphComplex

# We start by generating the encodings for the descendants of Gamma_{11}^2D and Gamma_{12}^2D
import itertools

X_graph_encodings = []
for (i1,j1,j2,k) in itertools.product([2,5], [1,4], [1,4], [3,6]):
    X_graph_encodings.append((0,1,4,j1,k,5,j2,i1,6))

for (i1,i2,j1,j2,k) in itertools.product([2,5], [2,5], [1,4], [1,4], [3,6]):
    X_graph_encodings.append((0,i1,4,j1,k,5,j2,i2,6))
    
# To move from the encodings of the graphs to the actual graphs, we use the next function. The definition splits the 
# encoding by vertex via ;, and then the targets by ,. A graph is returned on 3 Levi-Civita vertices, 3 corresponding Casimir
# vertices, 1 sink, and with edges (origin vertex, target vertex). 

def encoding_to_graph(encoding):
    targets = [encoding[0:3], encoding[3:6], encoding[6:9]]
    edges = sum([[(k+1,v) for v in t] for (k,t) in enumerate(targets)], [])
    return FormalityGraph(1, 6, edges)

X_graphs = [encoding_to_graph(e) for e in X_graph_encodings]
print('We have', len(X_graphs), 'graphs.\n')

# Below, we check the how many graphs that were created via the encodings are (non)isomorphic. It does so in the following way:
# for each formality graph in X_graphs, it computes the edges of the canonical form (canonical in the sense that isomorphic
# graphs return the same canonical form) of said formality graph. If a graph with these edges already appears in X_graphs_iso,
# the new (isomorphic) graph is not added to the list. If the graph does not yet appear, it is added to the list. The list is
# indexed by h, NOT by an index from 0,...,40. For this, unmute the line X_graphs_iso=list(X_graphs_iso.values())
X_graphs_iso={}
for g in X_graphs:
    h=tuple(g.canonical_form().edges())
    if not h in X_graphs_iso:
        X_graphs_iso[h]=g
X_graphs_iso=list(X_graphs_iso.values())
print ('Lemma 8: There are', len(X_graphs_iso), 'nonisomorphic graphs.\n')

# Create the differential polynomial ring. We are working in 3D, so we only have even coordinates x,y,z and the corresponding 
# odd coordinates xi[0], xi[1] and xi[2]. rho is exactly the rho in a 3D Nambu-determinant Poisson bracket,  
# (P(f,g)= rho d(f,g,a)/d(x,y,z)), and a is the Casimir.  Finally, max_differential_orders  tells the programme how many times 
# rho and a can be  differentiated. The maximum is stipulated by the graphs, as we cannot have double edges. Thus, the 
# maximum in degree of each rho vertex is 3. As we will be looking at [[P,X]], we add an extra +1 to the differential orders
# since taking the Schouten bracket with P introduces an extra derivative.
D3 = DifferentialPolynomialRing(QQ, ('rho','a'), ('x','y','z'), max_differential_orders=[3+1,3+1]) 
rho, a = D3.fibre_variables()
x,y,z = D3.base_variables()
even_coords = [x,y,z]

S3.<xi0,xi1,xi2> = SuperfunctionAlgebra(D3, D3.base_variables()) 
xi = S3.gens()
odd_coords = xi

# We now compute the vector fields corresponding to the graphs. E is the Euler vector field in the sink (vertex 0), and
# epsilon is the Levi-Civita tensor. Note that we have a Levi-Civita tensor at each of the vertices 1, 2, 3. We first compute
# the sign of each term appearing in the formulas, and then compute the differential polynomial.
X_vector_fields=[]
E = x*xi[0] + y*xi[1] + z*xi[2] 
epsilon = xi[0]*xi[1]*xi[2] 
for g in X_graphs:
    term = S3.zero()
    for index_choice in itertools.product(itertools.permutations(range(3)), repeat=3):
        sign = epsilon[index_choice[0]] * epsilon[index_choice[1]] * epsilon[index_choice[2]] 
        vertex_content = [E, S3(rho), S3(rho), S3(rho), S3(a), S3(a), S3(a)]
        for ((source, target), index) in zip(g.edges(), sum(map(list, index_choice), [])):
            vertex_content[target] = diff(vertex_content[target], even_coords[index])
        term += sign * prod(vertex_content)
    X_vector_fields.append(term)

# We check how many (if any) graphs evaluate to 0. 
zeros=X_vector_fields.count(0)
print('There are', zeros, 'graphs that evaluate to 0 under the morhpism from graphs to multivectors.')

# In case the are graphs that evaluate to 0, the line below shows which graphs do so (note that the counting starts from 1!).
print('These graphs are:')
for (k,X) in enumerate(X_vector_fields):
    if X==0:
        print('graph', k+1,)
print()

# The next part is to find out linear relations of the vector fields we just computed. We look at the monomials that appear
# in each xi[0], xi[1] and xi[2] parts of the vector field, and store them in X_monomial_basis. 
X_monomial_basis = [set([]) for i in range(3)]
for i in range(3):
    for X in X_vector_fields:
        X_monomial_basis[i] |= set(X[i].monomials())
X_monomial_basis = [list(b) for b in X_monomial_basis]
X_monomial_count = sum(len(b) for b in X_monomial_basis)

# Next, we use this monomial basis to create a matrix that identifies each vector field by the monomials that appear in it.
X_evaluation_matrix = matrix(QQ, X_monomial_count, len(X_vector_fields))
for i in range(len(X_vector_fields)):
    v = vector(QQ, X_monomial_count)
    index_shift = 0
    for j in range(3):
        f = X_vector_fields[i][j]
        for coeff, monomial in zip(f.coefficients(), f.monomials()):
            monomial_index = X_monomial_basis[j].index(monomial)
            v[index_shift + monomial_index] = coeff
        index_shift += len(X_monomial_basis[j])
    X_evaluation_matrix.set_column(i, v)

# We can now detect linear relation by computing the nullity of this matrix (Note that we already know of 7 linear relations
# as 7 of the 48 graphs are counted double via an isomorphism.
nullity = X_evaluation_matrix.right_nullity()
print('The vector fields have', nullity, 'linear relations among themselves.\n')

# Unmute the next line if you want to explicitly see the 28 linear relations among the vector fields
print('These relations are expressed by \n', X_evaluation_matrix.right_kernel().basis(), '\n')

# We now compute the tetrahedral flow. First, we create the Poisson bivector P and check that it satisfies [[P,P]]=0. 
P= (rho*epsilon).bracket(a)
if P.bracket(P)!=0:
    print('P is not a Poisson bivector. \n')

# Introduce the graph complex, and find the tetrahedron as a graph on 4 Poisson vertices and 6 edges in the cohomology. This 
# tetrahedron is unoriented, so we orient it. The next step is to make sure that the graph is built of wedges. This means 
# that there cannot be more than 2 outgoing edges at each vertex. This gives 2 graphs; one where 3 vertices have 2 outgoing
# edges, and one where 2 vertices have 2 outgoing edges and 2 vertices have 1 outgoing edge. 
# tetrahedron_oriented_filtered.show() gives a drawing of these graphs. 
# Finally, the bivector corresponding to the graph is computed.
GC=UndirectedGraphComplex(QQ, implementation='vector', sparse=True)
tetrahedron= GC.cohomology_basis(4,6)[0]
dGC=DirectedGraphComplex(QQ, implementation='vector')
tetrahedron_oriented= dGC(tetrahedron)
tetrahedron_oriented_filtered= tetrahedron_oriented.filter(max_out_degree=2)
#tetrahedron_oriented_filtered.show()
tetrahedron_operation= S3.graph_operation(tetrahedron_oriented_filtered)
Q_tetra= tetrahedron_operation(P, P, P, P) 
# print('The tetrahedral flow in 3D is', Q_tetra, '\n')

# Now that we have the tetrahedral flow, we see if we can create a vectorfield X from our previously computed 
# graphs-to-vectorfields such that [[P,X]]= Q_tetra. We first look which bivectors X_bivectors the vector fields become 
# after taking the Schouten bracket with P.
X_bivectors=[]
for X in X_vector_fields:
    X_bivectors.append(P.bracket(X))

# There are no nonzero (new) vector fields evaluated from 1 graph such that [[P,X_{graph}]]=0. 
zero_bivectors = X_bivectors.count(0)
print('There are', zero_bivectors, 'vector fields in X_vector_fields that evaluate to 0 bivectors after taking the Schouten bracket with P .')
print('These vector fields that evaluate to 0 are obtained from the graphs:')
for (k,X) in enumerate(X_bivectors):
    if X==0:
        print('graph', k+1,)
print()

# Now, we extract the monomials appearing in these bivectors (as well as in Q_tetra).
Q_monomial_basis={}
from itertools import combinations
for i,j in combinations(range(2),2):
    Q_monomial_basis[i,j]=set(Q_tetra[i,j].monomials())
    for P_X in X_bivectors:
        Q_monomial_basis[i,j]|= set(P_X[i,j].monomials())
        
Q_monomial_basis={idx: list(b) for idx, b in Q_monomial_basis.items()}
Q_monomial_index= {idx:{m:k for k,m in enumerate(b)} for idx, b in Q_monomial_basis.items()}
Q_monomial_count=sum(len(b) for b in Q_monomial_basis.values());

# We create the vector representation of Q_tetra in terms of the monomials.
Q_tetra_vector= vector(QQ, Q_monomial_count, sparse=True)
index_shift=0
for i,j in Q_monomial_basis:
    for coeff, monomial in Q_tetra[i,j]:
        monomial_index= Q_monomial_index[i,j][monomial]
        Q_tetra_vector[monomial_index+index_shift]=coeff
    index_shift+=len(Q_monomial_basis[i,j])

# We create the matrix that represents the X_bivectors in terms of the monomials.
X_bivector_evaluation_matrix= matrix(QQ, Q_monomial_count, len(X_bivectors), sparse=True)
for k in range(len(X_bivectors)):
    P_X=X_bivectors[k]
    v=vector(QQ,Q_monomial_count, sparse=True)
    index_shift=0
    for i,j in Q_monomial_basis:
        for coeff, monomial in P_X[i,j]:
            monomial_index=Q_monomial_index[i,j][monomial]
            v[monomial_index+index_shift]=coeff
        index_shift+=len(Q_monomial_basis[i,j])
    X_bivector_evaluation_matrix.set_column(k,v)

# We sovle the linear system.
X_solution=X_bivector_evaluation_matrix.solve_right(Q_tetra_vector)
print('Proposition 9: The solution on vector fields is given by', X_solution,' \n')

# Finally, we will check the homogeneous system. We look at the kernel of the X_bivector_evaluation_matrix and filter out the 
# nullity of the X_evaluation matrix.
X_cocycle_space= X_bivector_evaluation_matrix.right_kernel().quotient(X_evaluation_matrix.right_kernel())
X_cocycles=[X_cocycle_space.lift(v) for v in X_cocycle_space.basis()]
if all(X_bivector_evaluation_matrix*X_cocycle==0 for X_cocycle in X_cocycles)!= True:
    print('There is an error in the cocycle space.')
print('The space of solutions to the homogeneous system has dimension', X_cocycle_space.dimension(), )
print ('Proposition 10: These shifts are given by', X_cocycles, '\n')

# Let us write the vector fields as vector fields X such that [[P,X]]=0, rather than just the combination of graphs.
shifts_formulas = [sum(X_cocycle[j]*X_vector_fields[j] for j in range(len(X_vector_fields))) for X_cocycle in X_cocycles]

# We check that the shifts above is induced by the Hamiltonian vector field.
hamiltonian_encodings= [(1,2,3,0,2,3), (1,2,3,1,2,3), (1,2,3,0,1,3), (0,2,3,0,1,3), (0,2,3,1,2,3), (0,1,2,0,1,3), (0,1,2,1,2,3)]

# We create a new encoding to graph definition, as the Hamiltonians are graphs on 2 Levi-Civita vertices, 2 Casimir vertices 
# and no sink.
def hamiltonian_encoding_to_graph(encoding):
    targets = [encoding[0:3], encoding[3:6]]
    edges = sum([[(k+1,v) for v in t] for (k,t) in enumerate(targets)], [])
    return FormalityGraph(0, 4, edges)
hamiltonian_graphs = [encoding_to_graph(e) for e in hamiltonian_encodings]

# This computation is also slightly different; we do not have the Euler vector field since we do not have a sink, and as
# we have only 2 copies of the Poisson bivector, the index_choice for the sign is only on 2 repeats as well. Moreover, there
# are only 2 Levi-Civita vertices and 2 Casimir vertices to give vertex_content to. 
hamiltonian_formulas = []
epsilon = xi[0]*xi[1]*xi[2] # Levi-Civita tensor
for h in hamiltonian_graphs:
    term = S3.zero()
    for index_choice in itertools.product(itertools.permutations(range(3)), repeat=2):
        # TODO: Check that this way of evaluating is correct.
        sign = epsilon[index_choice[0]] * epsilon[index_choice[1]]
        vertex_content = [S3(rho), S3(rho), S3(a), S3(a)]
        for ((source, target), index) in zip(h.edges(), sum(map(list, index_choice), [])):
            vertex_content[target] = diff(vertex_content[target], even_coords[index])
        term += sign * prod(vertex_content)
    hamiltonian_formulas.append(term)

# We create the vector fields [[P,H]] from the Hamiltonians H.
hamiltonian_vector_fields=[]
for hamiltonian in hamiltonian_formulas:
    hamiltonian_vector_fields.append(P.bracket(hamiltonian))

# Create the basis of monomials.
hamiltonian_monomial_basis = {}
for j in range(len(shifts_formulas)):
    for i in range(3): 
        hamiltonian_monomial_basis[i] = set(shifts_formulas[j][i].monomials())
        for formula in hamiltonian_vector_fields:
            hamiltonian_monomial_basis[i] |= set(formula[i].monomials())

hamiltonian_monomial_basis = {idx: list(b) for idx, b in hamiltonian_monomial_basis.items()}
hamiltonian_monomial_index = {idx: {m : k for k, m in enumerate(b)} for idx, b in hamiltonian_monomial_basis.items()}
hamiltonian_monomial_count = sum(len(b) for b in hamiltonian_monomial_basis.values())

# Now, we write the shifts in terms of a vector of the monomials.
def shift_formula_to_vector(shift_formula):
        shift_vector = vector(QQ,hamiltonian_monomial_count, sparse=True)
        index_offset = 0
        for i in hamiltonian_monomial_basis:
            for coeff, monomial in shift_formula[i]:
                monomial_index = hamiltonian_monomial_index[i][monomial]
                shift_vector[monomial_index + index_offset] = coeff
            index_offset += len(hamiltonian_monomial_basis[i])
        return shift_vector

shifts_vectors = [shift_formula_to_vector(shift_formula) for shift_formula in shifts_formulas]

# Create the evaluation matrix for the Hamiltonian vector fields.
hamiltonian_evaluation_matrix = matrix(QQ,hamiltonian_monomial_count,len(hamiltonian_vector_fields),sparse=True)
for k in range(len(hamiltonian_vector_fields)):
    formula = hamiltonian_vector_fields[k] 
    v = vector(QQ, hamiltonian_monomial_count, sparse=True)
    index_shift = 0
    for i in hamiltonian_monomial_basis:
        for coeff, monomial in formula[i]:
            monomial_index = hamiltonian_monomial_index[i][monomial]
            v[monomial_index +index_shift] = coeff
        index_shift += len(hamiltonian_monomial_basis[i])
    hamiltonian_evaluation_matrix.set_column(k, v)

nullity=hamiltonian_evaluation_matrix.right_nullity()
kernel_basis= hamiltonian_evaluation_matrix.right_kernel().basis()
print('The nullity among the Hamiltonian vector fields is ', nullity, '')
# Note that the below is on the level of vector fields, not formulas. Unfortunately, the evaluation_matrix procedure only
# works from vector fields onward, and not for 0-vectors (formulas). You can explictly check that the linear relations ARE 
# already preserved on the level of formulas by simply evaluating hamiltonian_formulas[0]==hamiltonian_formulas[4] etc.
print('Explicitly, these are the linear combinations that evaluate to 0:', kernel_basis, '\n')

# We can now solve the shifts! Note that if (at least one) shift is NOT Hamiltonian, the code will break here as no solution 
# can be found.
shifts_solutions = [hamiltonian_evaluation_matrix.solve_right(shift_vector) for shift_vector in shifts_vectors]

# Write the solutions above from combinations of graphs to their vector field forms to check validity.
solution_formulas = [sum(shift_solution[i]*hamiltonian_vector_fields[i] for i in range(len(shift_solution))) for shift_solution in shifts_solutions]

if any(P.bracket(solution_formula) != 0 for solution_formula in solution_formulas) or solution_formulas != shifts_formulas:
    print('There is an error in computing the shifts.')
else:
    print('Theorem 12: The shifts in the cocycle space are Hamiltonian.')

for k, shift_solution in enumerate(shifts_solutions):
        print('The shift #', k+1, 'is the following linear combination of Hamiltonian vector fields:', shift_solution)
